# OpenAI Agents SDK - RunResult 분석

이 튜토리얼은 OpenAI Agents SDK에서 `RunResult` 객체를 활용하여 에이전트 실행 결과를 분석하고 모니터링하는 방법을 다룬다. 기본 속성 접근부터 심층 분석, 대화 관리, 프로덕션 모니터링까지 단계별로 학습한다.

In [ ]:
# 필요한 패키지 설치
!pip install openai-agents python-dotenv

---

## 1. RunResult 기초

이 챕터에서는 `RunResult` 객체의 필수 속성과 기본 사용법을 다룬다. 에이전트 응답에 접근하고 대화 흐름을 구축하는 기초 지식을 학습한다.

### 학습 내용

- `final_output`을 통한 에이전트 응답 접근
- 실행 메타데이터 분석 (에이전트 정보, 항목 수)
- `to_input_list()`를 사용한 대화 연속성 유지
- 토큰 사용량 및 API 비용 모니터링

### 환경 설정 및 도구 정의

먼저 필요한 모듈을 임포트하고 에이전트가 사용할 도구를 정의한다. 현재 시간을 반환하는 간단한 도구를 만들어 도구 호출이 포함된 RunResult를 분석할 수 있게 한다.

In [2]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool
from datetime import datetime

# 환경 변수 로드
load_dotenv()

MODEL = "gpt-4o-mini"

In [3]:
# 현재 시간을 반환하는 도구 정의
@function_tool
def get_current_time() -> str:
    """현재 시간을 반환한다."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("get_current_time 도구가 정의되었다.")

get_current_time 도구가 정의되었다.


### 에이전트 생성 및 실행

시간 관련 질문에 응답하는 에이전트를 생성하고 실행한다. `Runner.run_sync()`는 `RunResult` 객체를 반환하며, 이 객체를 통해 다양한 실행 정보에 접근할 수 있다.

In [5]:
# 에이전트 생성
agent = Agent(
    name="TimeKeeper",
    instructions="사용 가능한 도구를 사용하여 시간 관련 질문에 도움을 주세요.",
    tools=[get_current_time]
)

# 에이전트 실행
result = await Runner.run(agent, "지금 몇 시인가요?")

print("에이전트 실행 완료.")

에이전트 실행 완료.


### RunResult 기본 속성

`RunResult` 객체의 주요 속성을 살펴본다. `final_output`은 에이전트의 최종 응답 텍스트를 담고 있으며, `last_agent`는 마지막으로 실행된 에이전트의 정보를 제공한다. `new_items`는 실행 과정에서 생성된 항목들의 목록이고, `raw_responses`는 원시 API 응답을 담고 있다.

In [6]:
print("=== RunResult 기본 속성 ===")

# 주요 출력
print(f"\n[최종 응답]")
print(f"  {result.final_output}")

# 실행 메타데이터
print(f"\n[실행 메타데이터]")
print(f"  실행 에이전트: {result.last_agent.name}")
print(f"  생성된 총 항목 수: {len(result.new_items)}")
print(f"  원시 API 응답 수: {len(result.raw_responses)}")

=== RunResult 기본 속성 ===

[최종 응답]
  지금은 2025년 12월 7일 23시 38분입니다.

[실행 메타데이터]
  실행 에이전트: TimeKeeper
  생성된 총 항목 수: 3
  원시 API 응답 수: 2


### 대화 연속성을 위한 to_input_list()

`to_input_list()` 메서드는 현재까지의 대화 기록을 다음 호출에 전달할 수 있는 형식으로 변환한다. 이 메서드를 사용하면 멀티턴 대화에서 컨텍스트를 유지할 수 있다.

In [7]:
# 대화 연속을 위한 입력 리스트
conversation_history = result.to_input_list()

print("=== 대화 연속성 ===")
print(f"다음 턴을 위한 메시지 수: {len(conversation_history)}")
print(f"\n대화 기록 구조:")
for i, item in enumerate(conversation_history[:5]):  # 처음 5개만 표시
    if isinstance(item, dict):
        role = item.get('role', 'unknown')
        content = str(item.get('content', ''))[:50]
        print(f"  {i+1}. [{role}] {content}{'...' if len(str(item.get('content', ''))) > 50 else ''}")
    else:
        print(f"  {i+1}. {type(item).__name__}")

=== 대화 연속성 ===
다음 턴을 위한 메시지 수: 4

대화 기록 구조:
  1. [user] 지금 몇 시인가요?
  2. [unknown] 
  3. [unknown] 
  4. [assistant] [{'annotations': [], 'text': '지금은 2025년 12월 7일 23시...


### 토큰 사용량 분석

`raw_responses`를 통해 각 API 호출의 토큰 사용량을 확인할 수 있다. 이 정보는 비용 관리와 성능 최적화에 필수적이다.

In [8]:
# 사용량 분석
print("=== 토큰 사용량 분석 ===")
if result.raw_responses:
    total_input = 0
    total_output = 0
    
    for i, response in enumerate(result.raw_responses):
        usage = response.usage
        total_input += usage.input_tokens
        total_output += usage.output_tokens
        print(f"턴 {i+1}: 입력 {usage.input_tokens}, 출력 {usage.output_tokens}")
    
    print(f"\n총계: 입력 {total_input}, 출력 {total_output}, 합계 {total_input + total_output}")
else:
    print("원시 응답이 없다.")

=== 토큰 사용량 분석 ===
턴 1: 입력 57, 출력 12
턴 2: 입력 90, 출력 23

총계: 입력 147, 출력 35, 합계 182


---

## 2. RunResult 심층 분석

이 챕터에서는 `RunResult` 객체를 종합적으로 분석하는 방법을 다룬다. 디버깅 함수를 구축하고, 실행 흐름을 이해하며, 비용을 추정하는 방법을 학습한다.

### 학습 내용

- 에이전트 실행을 분석하는 디버깅 함수 구축
- new_items 분석을 통한 실행 흐름 이해
- 토큰 사용량 추적 및 비용 추정
- 성능 모니터링 및 최적화 인사이트

In [9]:
import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool
from datetime import datetime

### 종합 분석 함수 정의

프로덕션 환경에서 디버깅과 모니터링에 활용할 수 있는 종합 분석 함수를 정의한다. 이 함수는 기본 속성, 항목 분석, 토큰 사용량, 비용 추정, 대화 관리 정보를 한 번에 출력한다.

In [10]:
def analyze_runresult(result, title="RunResult 분석"):
    """
    RunResult 객체를 종합적으로 분석하는 함수이다.
    디버깅과 모니터링에 활용할 수 있다.
    
    Args:
        result: 분석할 RunResult 객체
        title: 분석 결과의 제목
    """
    
    print(f"\n{'='*60}")
    print(f"{title.upper()}")
    print('='*60)
    
    # 기본 속성
    print("[기본 속성]")
    print(f"   최종 출력 타입: {type(result.final_output)}")
    print(f"   최종 출력 길이: {len(str(result.final_output))} 문자")
    print(f"   마지막 에이전트: {result.last_agent.name}")
    print(f"   총 새 항목 수: {len(result.new_items)}")
    print(f"   원시 응답 수: {len(result.raw_responses)}")
    
    # 항목 분석 - 무슨 일이 일어났는지 파악
    if result.new_items:
        print(f"\n[새 항목 분석]")
        item_types = {}
        for item in result.new_items:
            item_type = item.type
            item_types[item_type] = item_types.get(item_type, 0) + 1
        
        for item_type, count in item_types.items():
            print(f"   {item_type}: {count}개")
    
    # 토큰 사용량 및 비용 추적
    if result.raw_responses:
        print(f"\n[토큰 사용량]")
        total_input = 0
        total_output = 0
        
        for i, response in enumerate(result.raw_responses):
            usage = response.usage
            total_input += usage.input_tokens
            total_output += usage.output_tokens
            print(f"   턴 {i+1}: 입력 {usage.input_tokens}, 출력 {usage.output_tokens}")
        
        print(f"   총계: 입력 {total_input}, 출력 {total_output}")
        
        # 비용 추정 (GPT-4o 기준 예시 요금)
        cost_per_1k_input = 0.0025
        cost_per_1k_output = 0.01
        estimated_cost = (total_input / 1000 * cost_per_1k_input) + (total_output / 1000 * cost_per_1k_output)
        print(f"   예상 비용: ${estimated_cost:.6f}")
    
    # 대화 연속성 준비 상태
    print(f"\n[대화 관리]")
    input_list = result.to_input_list()
    print(f"   입력 리스트 길이: {len(input_list)}")
    print(f"   연속 대화 준비: {'완료' if input_list else '미완료'}")

print("analyze_runresult 함수가 정의되었다.")

analyze_runresult 함수가 정의되었다.


### 도구가 포함된 에이전트 테스트

도구를 사용하는 에이전트를 만들고 다양한 쿼리에 대한 실행 결과를 분석한다. 도구 호출이 있는 경우와 없는 경우의 차이를 비교할 수 있다.

In [11]:
# 현재 시간을 반환하는 도구 정의
@function_tool
def get_current_time() -> str:
    """현재 시간을 반환한다."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# 도구가 있는 에이전트 생성
agent = Agent(
    name="AnalyticsAgent",
    instructions="당신은 도구를 사용할 수 있는 도움이 되는 어시스턴트입니다. 시간에 대해 질문받으면 시간 도구를 사용하세요.",
    tools=[get_current_time]
)

print(f"에이전트 '{agent.name}'가 생성되었다.")

에이전트 'AnalyticsAgent'가 생성되었다.


In [12]:
# 간단한 쿼리 테스트
print("간단한 쿼리 테스트 중...")
result1 = await Runner.run(agent, "지금 몇 시인가요?")
analyze_runresult(result1, "간단한 시간 쿼리")

print(f"\n에이전트 응답: {result1.final_output}")

간단한 쿼리 테스트 중...

간단한 시간 쿼리
[기본 속성]
   최종 출력 타입: <class 'str'>
   최종 출력 길이: 28 문자
   마지막 에이전트: AnalyticsAgent
   총 새 항목 수: 3
   원시 응답 수: 2

[새 항목 분석]
   tool_call_item: 1개
   tool_call_output_item: 1개
   message_output_item: 1개

[토큰 사용량]
   턴 1: 입력 73, 출력 12
   턴 2: 입력 106, 출력 23
   총계: 입력 179, 출력 35
   예상 비용: $0.000798

[대화 관리]
   입력 리스트 길이: 4
   연속 대화 준비: 완료

에이전트 응답: 지금은 2025년 12월 7일 23시 38분입니다.


In [13]:
# 복잡한 쿼리 테스트
print("복잡한 쿼리 테스트 중...")
result2 = await Runner.run(
    agent, 
    "현재 시간을 알려주시고, 시간 관리가 왜 중요한지 설명해주세요."
)
analyze_runresult(result2, "도구 사용이 포함된 복잡한 쿼리")

print(f"\n에이전트 응답: {result2.final_output}")

복잡한 쿼리 테스트 중...

도구 사용이 포함된 복잡한 쿼리
[기본 속성]
   최종 출력 타입: <class 'str'>
   최종 출력 길이: 315 문자
   마지막 에이전트: AnalyticsAgent
   총 새 항목 수: 3
   원시 응답 수: 2

[새 항목 분석]
   tool_call_item: 1개
   tool_call_output_item: 1개
   message_output_item: 1개

[토큰 사용량]
   턴 1: 입력 81, 출력 30
   턴 2: 입력 111, 출력 169
   총계: 입력 192, 출력 199
   예상 비용: $0.002470

[대화 관리]
   입력 리스트 길이: 4
   연속 대화 준비: 완료

에이전트 응답: 현재 시간은 2025년 12월 7일 23시 38분입니다.

시간 관리가 중요한 이유는 다음과 같습니다:
1. 효율성 향상: 시간을 잘 관리하면 더 짧은 시간에 더 많은 일을 할 수 있습니다.
2. 스트레스 감소: 일정을 체계적으로 계획하면 마감이나 과중한 업무로 인한 스트레스를 줄일 수 있습니다.
3. 목표 달성: 시간을 체계적으로 분배하여 목표를 체계적으로 달성할 수 있습니다.
4. 일-생활 균형: 업무 외에 개인적인 시간도 확보할 수 있어 삶의 질이 올라갑니다.

시간을 잘 관리하는 습관을 들이면 일상과 업무 모두에서 긍정적인 변화를 경험할 수 있습니다!


---

## 3. RunResult와 대화 관리

이 챕터에서는 `RunResult`를 활용하여 멀티턴 대화를 관리하는 방법을 다룬다. 여러 턴에 걸쳐 대화 기록을 구축하고, 컨텍스트를 유지하며, 대화 성장을 분석하는 방법을 학습한다.

### 학습 내용

- 여러 턴에 걸친 대화 기록 구축
- `to_input_list()`를 사용한 상호작용 간 컨텍스트 유지
- 시간에 따른 토큰 사용량 및 대화 성장 분석
- 멀티턴 대화를 위한 대화 상태 관리

In [14]:
import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner, function_tool
from datetime import datetime

In [15]:
# 도구 정의
@function_tool
def get_current_time() -> str:
    """현재 시간을 반환한다."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print("도구가 정의되었다.")

도구가 정의되었다.


### 멀티턴 대화 분석 함수

여러 턴에 걸친 대화를 수행하면서 각 턴의 토큰 사용량과 대화 길이 변화를 추적하는 함수를 정의한다. 이를 통해 대화가 진행됨에 따라 컨텍스트가 어떻게 성장하는지 파악할 수 있다.

In [16]:
async def conversation_with_analysis():
    """
    멀티턴 대화를 수행하면서 각 턴을 분석하는 함수이다.
    """
    agent = Agent(
        name="Assistant",
        instructions="당신은 도움이 되는 어시스턴트이며 대화를 기억합니다.",
        tools=[get_current_time]
    )
    
    # 대화 시나리오
    conversations = [
        "지금 몇 시인가요?",
        "그 시간을 기억해주세요. 아까 몇 시였죠?",
        "그때로부터 얼마나 지났나요?"
    ]
    
    conversation_history = []
    turn_stats = []  # 각 턴의 통계 저장
    
    print("=== 멀티턴 대화 분석 ===")
    
    for i, user_input in enumerate(conversations):
        print(f"\n--- 턴 {i+1} ---")
        print(f"사용자: {user_input}")
        
        # 대화 기록과 함께 입력 구축
        if conversation_history:
            full_input = conversation_history + [{"role": "user", "content": user_input}]
        else:
            full_input = user_input
        
        # 실행 및 분석
        result = await Runner.run(agent, full_input)
        print(f"어시스턴트: {result.final_output}")
        
        # 대화 기록 업데이트
        conversation_history = result.to_input_list()
        
        # 턴 통계 수집
        if result.raw_responses:
            usage = result.raw_responses[0].usage
            stats = {
                "turn": i + 1,
                "input_tokens": usage.input_tokens,
                "output_tokens": usage.output_tokens,
                "total_tokens": usage.total_tokens,
                "conversation_length": len(conversation_history)
            }
            turn_stats.append(stats)
            
            print(f"토큰: 입력 {usage.input_tokens}, 출력 {usage.output_tokens}")
        
        print(f"대화 길이: {len(conversation_history)} 메시지")
    
    return turn_stats, conversation_history

In [17]:
# 대화 분석 실행
turn_stats, final_history = await conversation_with_analysis()

=== 멀티턴 대화 분석 ===

--- 턴 1 ---
사용자: 지금 몇 시인가요?
어시스턴트: 현재 시간은 2025년 12월 7일 23시 39분입니다.
토큰: 입력 58, 출력 12
대화 길이: 4 메시지

--- 턴 2 ---
사용자: 그 시간을 기억해주세요. 아까 몇 시였죠?
어시스턴트: 네, 방금 전에 현재 시간이 2025년 12월 7일 23시 39분이라고 말씀드렸습니다. 앞으로도 이 시간을 기억하고 있을게요! 추가로 궁금한 점 있으신가요?
토큰: 입력 132, 출력 52
대화 길이: 6 메시지

--- 턴 3 ---
사용자: 그때로부터 얼마나 지났나요?
어시스턴트: 지금은 2025년 12월 7일 23시 39분 8초입니다. 아까(23시 39분 3초)로부터 약 5초가 지났습니다.
토큰: 입력 199, 출력 12
대화 길이: 10 메시지


### 대화 성장 분석

수집된 통계를 바탕으로 대화가 진행됨에 따라 토큰 사용량이 어떻게 변화하는지 분석한다. 대화가 길어질수록 컨텍스트가 누적되어 입력 토큰이 증가하는 패턴을 확인할 수 있다.

In [18]:
print("\n=== 대화 성장 분석 ===")
print(f"\n{'턴':<5} {'입력 토큰':<12} {'출력 토큰':<12} {'총 토큰':<12} {'대화 길이':<12}")
print("-" * 55)

for stats in turn_stats:
    print(f"{stats['turn']:<5} {stats['input_tokens']:<12} {stats['output_tokens']:<12} {stats['total_tokens']:<12} {stats['conversation_length']:<12}")

# 총계 계산
total_input = sum(s['input_tokens'] for s in turn_stats)
total_output = sum(s['output_tokens'] for s in turn_stats)
total_all = sum(s['total_tokens'] for s in turn_stats)

print("-" * 55)
print(f"{'총계':<5} {total_input:<12} {total_output:<12} {total_all:<12}")


=== 대화 성장 분석 ===

턴     입력 토큰        출력 토큰        총 토큰         대화 길이       
-------------------------------------------------------
1     58           12           70           4           
2     132          52           184          6           
3     199          12           211          10          
-------------------------------------------------------
총계    389          76           465         


In [19]:
# 대화 기록 구조 분석
print("\n=== 최종 대화 기록 구조 ===")
for i, item in enumerate(final_history):
    if isinstance(item, dict):
        role = item.get('role', 'unknown')
        content = str(item.get('content', ''))[:60]
        print(f"{i+1}. [{role}] {content}{'...' if len(str(item.get('content', ''))) > 60 else ''}")
    else:
        print(f"{i+1}. [{type(item).__name__}]")

print(f"\n대화가 진행됨에 따라 컨텍스트가 누적되어 입력 토큰이 증가하는 것을 확인할 수 있다.")


=== 최종 대화 기록 구조 ===
1. [user] 지금 몇 시인가요?
2. [unknown] 
3. [unknown] 
4. [assistant] [{'annotations': [], 'text': '현재 시간은 2025년 12월 7일 23시 39분입니다...
5. [user] 그 시간을 기억해주세요. 아까 몇 시였죠?
6. [assistant] [{'annotations': [], 'text': '네, 방금 전에 현재 시간이 2025년 12월 7일 2...
7. [user] 그때로부터 얼마나 지났나요?
8. [unknown] 
9. [unknown] 
10. [assistant] [{'annotations': [], 'text': '지금은 2025년 12월 7일 23시 39분 8초입니다...

대화가 진행됨에 따라 컨텍스트가 누적되어 입력 토큰이 증가하는 것을 확인할 수 있다.


---

## RunResult 프로덕션 모니터링

이 챕터에서는 프로덕션 환경에서 `RunResult`를 활용한 모니터링과 분석 방법을 다룬다. 분석 대시보드용 메트릭 내보내기, 토큰 사용량/도구 사용량/성능 추적, 디버깅과 최적화를 위한 구조화된 로그 생성 방법을 학습한다.

### 학습 내용

- 분석 대시보드용 대화 메트릭 내보내기
- 토큰 사용량, 도구 사용량, 성능 메트릭 추적
- 프로덕션 에이전트 배포를 위한 모니터링 함수 구축
- 디버깅 및 최적화를 위한 구조화된 로그 생성

In [20]:
import os
import json
import asyncio
from dotenv import load_dotenv
from datetime import datetime
from agents import Agent, Runner, function_tool

### 메트릭 내보내기 함수

프로덕션 모니터링을 위한 메트릭 내보내기 함수를 정의한다. 이 함수는 `RunResult`에서 토큰 사용량, 도구 호출 정보, 대화 길이 등 다양한 메트릭을 추출하여 구조화된 형식으로 반환한다. 이 데이터는 분석 대시보드, 로깅 시스템, 모니터링 도구에 전송할 수 있다.

In [21]:
def export_conversation_metrics(result, user_id=None, session_id=None):
    """
    분석 및 모니터링을 위한 대화 데이터를 내보내는 함수이다.
    
    Args:
        result: 분석할 RunResult 객체
        user_id: 선택적 사용자 식별자
        session_id: 선택적 세션 식별자
    
    Returns:
        구조화된 메트릭 딕셔너리
    """
    
    # 토큰 사용량 계산
    total_input = sum(r.usage.input_tokens for r in result.raw_responses)
    total_output = sum(r.usage.output_tokens for r in result.raw_responses)
    
    # 도구 사용량 분석
    tool_calls = [item for item in result.new_items if item.type == "tool_call_item"]
    
    # 비용 추정 (GPT-4o 기준)
    cost_per_1k_input = 0.0025
    cost_per_1k_output = 0.01
    estimated_cost = (total_input / 1000 * cost_per_1k_input) + (total_output / 1000 * cost_per_1k_output)
    
    metrics = {
        "timestamp": datetime.now().isoformat(),
        "user_id": user_id,
        "session_id": session_id,
        "agent_name": result.last_agent.name,
        "conversation_length": len(result.to_input_list()),
        "tokens": {
            "input": total_input,
            "output": total_output,
            "total": total_input + total_output
        },
        "cost": {
            "estimated_usd": round(estimated_cost, 6)
        },
        "tool_usage": {
            "tools_called": len(tool_calls),
            "total_items": len(result.new_items)
        },
        "response": {
            "length": len(result.final_output),
            "preview": result.final_output[:100] + "..." if len(result.final_output) > 100 else result.final_output
        }
    }
    
    return metrics

print("export_conversation_metrics 함수가 정의되었다.")

export_conversation_metrics 함수가 정의되었다.


### 모니터링 시스템 테스트

에이전트를 실행하고 메트릭을 내보내어 모니터링 시스템이 어떻게 작동하는지 테스트한다.

In [22]:
# 도구 정의
@function_tool
def get_current_time() -> str:
    """현재 시간을 반환한다."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

@function_tool
def calculate(expression: str) -> str:
    """수학 표현식을 계산한다."""
    try:
        result = eval(expression)
        return str(result)
    except Exception as e:
        return f"오류: {str(e)}"

print("도구가 정의되었다.")

도구가 정의되었다.


In [23]:
async def monitored_conversation():
    """
    모니터링이 적용된 대화를 수행하는 함수이다.
    """
    agent = Agent(
        name="MonitoredAgent", 
        instructions="당신은 도움이 되는 어시스턴트입니다. 필요한 경우 도구를 사용하세요.",
        tools=[get_current_time, calculate]
    )
    
    # 대화 실행
    result = await Runner.run(
        agent, 
        "안녕하세요! 지금 시간이 몇 시인지, 그리고 15 * 24가 얼마인지 알려주세요."
    )
    
    # 메트릭 내보내기
    metrics = export_conversation_metrics(
        result, 
        user_id="user_123",
        session_id="session_abc"
    )
    
    return result, metrics

# 실행
result, metrics = await monitored_conversation()

In [24]:
# 메트릭 출력
print("=== 대화 메트릭 ===")
print(json.dumps(metrics, indent=2, ensure_ascii=False))

=== 대화 메트릭 ===
{
  "timestamp": "2025-12-07T23:39:21.680971",
  "user_id": "user_123",
  "session_id": "session_abc",
  "agent_name": "MonitoredAgent",
  "conversation_length": 6,
  "tokens": {
    "input": 206,
    "output": 83,
    "total": 289
  },
  "cost": {
    "estimated_usd": 0.001345
  },
  "tool_usage": {
    "tools_called": 2,
    "total_items": 5
  },
  "response": {
    "length": 61,
    "preview": "안녕하세요! 지금 시간은 2025년 12월 7일 23시 39분입니다. 그리고 15 곱하기 24는 360입니다."
  }
}


In [25]:
# 에이전트 응답 출력
print("\n=== 에이전트 응답 ===")
print(result.final_output)


=== 에이전트 응답 ===
안녕하세요! 지금 시간은 2025년 12월 7일 23시 39분입니다. 그리고 15 곱하기 24는 360입니다.


### 배치 모니터링 시뮬레이션

여러 사용자의 대화를 시뮬레이션하고 집계 통계를 생성하는 예제이다. 프로덕션 환경에서 여러 사용자의 사용 패턴을 분석할 때 유용하다.

In [26]:
async def batch_monitoring_demo():
    """
    여러 대화에 대한 배치 모니터링을 시연하는 함수이다.
    """
    agent = Agent(
        name="BatchAgent", 
        instructions="당신은 간결하게 답변하는 어시스턴트입니다."
    )
    
    # 다양한 사용자 쿼리 시뮬레이션
    queries = [
        ("user_001", "파이썬이란 무엇인가요?"),
        ("user_002", "머신러닝을 간단히 설명해주세요."),
        ("user_003", "API란 무엇인가요?")
    ]
    
    all_metrics = []
    
    print("=== 배치 모니터링 데모 ===")
    
    for user_id, query in queries:
        result = await Runner.run(agent, query)
        metrics = export_conversation_metrics(result, user_id=user_id)
        all_metrics.append(metrics)
        
        print(f"\n[{user_id}] {query}")
        print(f"  응답 길이: {metrics['response']['length']} 문자")
        print(f"  토큰: {metrics['tokens']['total']}")
        print(f"  비용: ${metrics['cost']['estimated_usd']}")
    
    return all_metrics

all_metrics = await batch_monitoring_demo()

=== 배치 모니터링 데모 ===

[user_001] 파이썬이란 무엇인가요?
  응답 길이: 108 문자
  토큰: 94
  비용: $0.000663

[user_002] 머신러닝을 간단히 설명해주세요.
  응답 길이: 79 문자
  토큰: 80
  비용: $0.000515

[user_003] API란 무엇인가요?
  응답 길이: 211 문자
  토큰: 124
  비용: $0.000992


In [27]:
# 집계 통계 계산
print("\n=== 집계 통계 ===")

total_tokens = sum(m['tokens']['total'] for m in all_metrics)
total_cost = sum(m['cost']['estimated_usd'] for m in all_metrics)
avg_response_length = sum(m['response']['length'] for m in all_metrics) / len(all_metrics)

print(f"총 대화 수: {len(all_metrics)}")
print(f"총 토큰 사용량: {total_tokens}")
print(f"총 예상 비용: ${total_cost:.6f}")
print(f"평균 응답 길이: {avg_response_length:.0f} 문자")
print(f"대화당 평균 토큰: {total_tokens / len(all_metrics):.0f}")
print(f"대화당 평균 비용: ${total_cost / len(all_metrics):.6f}")


=== 집계 통계 ===
총 대화 수: 3
총 토큰 사용량: 298
총 예상 비용: $0.002170
평균 응답 길이: 133 문자
대화당 평균 토큰: 99
대화당 평균 비용: $0.000723


### 모니터링 데이터 활용 방안

수집된 메트릭을 활용할 수 있는 다양한 방안을 정리한다.


1. 비용 관리
   - 일일/월별 토큰 사용량 추적
   - 사용자별 비용 분석
   - 비용 이상 감지 알림 설정

2. 성능 최적화
   - 응답 시간 분석
   - 도구 호출 패턴 파악
   - 병목 지점 식별

3. 사용자 경험 개선
   - 자주 묻는 질문 분석
   - 실패 패턴 파악
   - 만족도 지표 추적

4. 시스템 건강 모니터링
   - 에러율 추적
   - API 응답 시간 모니터링
   - 용량 계획 수립

---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 `RunResult` 객체를 활용한 분석과 모니터링 방법을 다루었다. 다음과 같은 내용을 학습하였다:

1. **RunResult 기초**: final_output, last_agent, new_items, raw_responses 등 기본 속성
2. **대화 연속성**: to_input_list()를 사용한 멀티턴 대화 관리
3. **심층 분석**: 항목 분석, 토큰 사용량 추적, 비용 추정
4. **대화 관리**: 대화 성장 분석, 컨텍스트 누적 패턴 이해
5. **프로덕션 모니터링**: 메트릭 내보내기, 배치 분석, 집계 통계

이러한 패턴을 적용하면 에이전트의 동작을 깊이 이해하고, 비용을 관리하며, 프로덕션 환경에서 효과적인 모니터링 시스템을 구축할 수 있다.